# Pràctica 3 - PLH

### Realitzada pels alumnes Lluc Furriols i Pau Prat Moreno

In [1]:
'''
import os 
f = open("/dev/null", "w")
os.dup2(f.fileno(), 2)
f.close()

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
'''

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Importació de llibreries

In [2]:
import nltk
nltk.download('punkt', quiet=True) # Tokenitzador
nltk.download('averaged_perceptron_tagger', quiet=True) # Etiquetador POS
nltk.download('maxent_ne_chunker', quiet=True) # Etiquetador Entitats Anomenades
nltk.download('words', quiet=True)

True

## Carreguem les dades

In [3]:
nltk.download('conll2002')
from nltk.corpus import conll2002

train_esp = conll2002.iob_sents('esp.train') # Train, 
val_esp = conll2002.iob_sents('esp.testa') # Val
test_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
val_ned = conll2002.iob_sents('ned.testa') # Val
test_ned = conll2002.iob_sents('ned.testb') # Test

[nltk_data] Downloading package conll2002 to /Users/pau/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [4]:
train_esp

[[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')], [('-', 'Fg', 'O')], ...]

In [42]:
def get_token(sequence):
    """
    Retorna una llista de tokens.
    """
    return [[(token) for token, pos, entity in sentence] for sentence in sequence]

def get_token_POS(sequence):
    """
    Retorna una llista de tokens i el seu POS tag.
    """
    return [[(token, pos) for token, pos, entity in sentence] for sentence in sequence]

def get_token_entity(sequence):
    """
    Retorna una llista de tokens i les seves entitats (BIOES...).
    """
    return [[(token, entity) for token, pos, entity in sentence] for sentence in sequence]

# First execution with no modifications

In [6]:
from nltk.tag import CRFTagger
import pycrfsuite

ct = CRFTagger(feature_func=None)

# Train and test sets without the postag
train_esp_first = get_token_entity(train_esp)
test_esp_first = get_token_entity(test_esp)
print("Quina forma tenen les nostres dades d'entrenament: ",train_esp_first[0])

ct.train(train_esp_first, 'model.crf.tagger')

Quina forma tenen les nostres dades d'entrenament:  [('Melbourne', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B-ORG'), (')', 'O'), ('.', 'O')]


###  Probar el model en el conjunt de test

In [7]:
# Predir les entitats del conjunt de test
y_pred = ct.tag_sents(get_token(test_esp))
print(y_pred[:2])

[[('La', 'B-LOC'), ('Coruña', 'I-LOC'), (',', 'O'), ('23', 'O'), ('may', 'O'), ('(', 'O'), ('EFECOM', 'B-ORG'), (')', 'O'), ('.', 'O')], [('-', 'O')]]


In [9]:
# Les entitats reals del conjunt de test
y_real = get_token_entity(test_esp)
print(y_real[:2])

[[('La', 'B-LOC'), ('Coruña', 'I-LOC'), (',', 'O'), ('23', 'O'), ('may', 'O'), ('(', 'O'), ('EFECOM', 'B-ORG'), (')', 'O'), ('.', 'O')], [('-', 'O')]]


In [10]:
def extract_entities(tagged_words):
    entities = []
    current_entity = []
    current_type = None
    current_start_index = None  # Afegim una variable per guardar l'índex d'inici

    for index, (word, tag) in enumerate(tagged_words):
        if tag.startswith('B-'):  # Començament d'una nova entitat
            if current_entity:  # Si hi havia una entitat en construcció, l'afegim abans de començar la nova
                entities.append((current_start_index, index - 1, current_type))
            current_entity = [word]  # Comencem una nova entitat
            current_start_index = index  # Guardem l'índex d'inici de l'entitat actual
            current_type = tag[2:]  # Guardem el tipus d'entitat sense el prefix B-
        elif tag.startswith('I-') and current_type == tag[2:]:  # Continuació de la mateixa entitat
            current_entity.append(word)
        else:  # Si no és una continuació de la mateixa entitat o és 'O'
            if current_entity:  # Finalitzem l'entitat actual si n'hi ha una
                entities.append((current_start_index, index - 1, current_type))
                current_entity = []
                current_type = None
            if tag == 'O':
                continue
            else:  # Codificació IO o canvi d'entitat amb I-
                current_entity = [word]
                current_start_index = index
                current_type = tag[2:]  # Possible en cas de codificació IO

    # Assegurar-se d'afegir l'última entitat si la llista no acaba en 'O'
    if current_entity:
        entities.append((current_start_index, index, current_type))

    return entities


def evaluate_entities(y_test, y_pred, print_errors=False):
    """
    Evaluate the performance of a named entity recognition model.

    This function calculates the precision, recall, and F1 score of the model's predictions. It only evaluates predictions in terms of entities, not individual tokens.
    It also optionally prints the sentences where the model made errors.

    Parameters:
    y_test (list): The true labels for the test data. Each element is a list of tuples, where each tuple contains a token and its true label.
    y_pred (list): The predicted labels for the test data. Each element is a list of tuples, where each tuple contains a token and its predicted label.
    print_errors (bool, optional): Whether to print the sentences where the model made errors. Defaults to False.

    Returns:
    precision (float): The precision of the model's predictions.
    recall (float): The recall of the model's predictions.
    f1_score (float): The F1 score of the model's predictions.
    """

    total_entities = 0
    correct_entities = 0

    for sent_test, sent_pred in zip(y_test, y_pred):
        true_entities = extract_entities(sent_test)
        pred_entities = extract_entities(sent_pred)
        
        # Comptar entitats reals
        entities_sentence = len(true_entities)
        # Entitats correctament predites
        entities_predicted = len([e for e in true_entities if e in pred_entities])

        # Portem el compte de totes les entitats
        total_entities += entities_sentence
        # Portem el compte de les entitats correctament predites
        correct_entities += entities_predicted

        if print_errors:
            if entities_sentence != entities_predicted:
                print('Real sentence:', sent_test)
                print('Predicted:', sent_pred)
                print('Difference in:', (set(true_entities) - set(pred_entities)))
                print()

        
    if total_entities == 0:
        return 0, 0, 0, 0

    precision = correct_entities / total_entities
    recall = correct_entities / total_entities
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1_score


precision, recall, f1_score = evaluate_entities(y_real, y_pred, print_errors=False)
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1 Score: {f1_score:.6f}')


Precision: 0.706378
Recall: 0.706378
F1 Score: 0.706378


## Canviar feature functions i codificacio

In [11]:
import string

class FeatureGetter:
    """
    Aquesta classe s'utilitza per obtenir diferents característiques d'un token de text.
    """
    def __init__(self):
        pass

    def has_capitalization(self, token):
        return any(char.isupper() for char in token)

    def has_digit(self, token):
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        return token[-n:] if len(token) > n else token

    def all_caps(self, token):
        return token.isupper()

    def is_capitalized(self, token):
        return token[0].isupper()

    def get_features(self, tokens, index, add_prefix_suffix=True):
        token = tokens[index]
        token = str(token)
        features = {
            'bias': 1.0,
            'has_capitalization': self.has_capitalization(token),
            'has_digit': self.has_digit(token),
            'has_punctuation': self.has_punctuation(token),
            'all_caps': self.all_caps(token),
            'is_capitalized': self.is_capitalized(token),
        }

        if add_prefix_suffix:
            features['prefix'] = self.get_prefix(token)
            features['suffix'] = self.get_suffix(token)

        return features

# Exemple d'ús:
feature_getter = FeatureGetter()
tokens = ['Barcelona', 'is', 'beautiful']
token_features = feature_getter.get_features(tokens, 0)
print(token_features)

{'bias': 1.0, 'has_capitalization': True, 'has_digit': False, 'has_punctuation': False, 'all_caps': False, 'is_capitalized': True, 'prefix': 'Bar', 'suffix': 'ona'}


In [39]:
from nltk.tag import CRFTagger
import re
import string


class FeatureGetter:
    def __init__(self):
        self._pattern = re.compile(r"\d")

    def has_digit(self, token):
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        return token[-n:] if len(token) > n else token

    def get_features(self, tokens, index):
        token = tokens[index]
        features = ["WORD_" + token]
        if token[0].isupper():
            features.append("CAPITALIZATION")
        if self.has_digit(token):
            features.append("HAS_NUM")
        if self.has_punctuation(token):
            features.append("PUNCTUATION")
        features.extend(["SUF_" + self.get_suffix(token, n) for n in range(1, 4) if len(token) >= n])
        features.extend(["PRE_" + self.get_prefix(token, n) for n in range(1, 4) if len(token) >= n])
        if index > 0:
            prev_token = tokens[index - 1]
            features.append("PREV_WORD_" + prev_token)
        if index < len(tokens) - 1:
            next_token = tokens[index + 1]
            features.append("NEXT_WORD_" + next_token)
        return features

# Creació de dades de prova
train_data = [
    [('University', 'Noun'), ('is', 'Verb'), ('a', 'Det'), ('Cai', 'gay'), ('good', 'Adj'), ('place', 'Noun')],
    [('Dog', 'Noun'), ('eats', 'Verb'), ('meat', 'Noun')]
]

# Instància del CRFTagger amb la funció de característiques personalitzada
feature_getter = FeatureGetter()
ct = CRFTagger(feature_func=lambda tokens, index: feature_getter.get_features(tokens, index))

# Entrenament del model
ct.train(train_data, 'model.crf.tagger')

# Previsió en un nou conjunt de dades
test_sentences = [['University', 'is', 'good'], ['Cat', 'eats', 'meat'], ['Cai']]
tagged_sentences = ct.tag_sents(test_sentences)

# Imprimir els resultats etiquetats
for sentence in tagged_sentences:
    print(sentence)


[('University', 'Noun'), ('is', 'Verb'), ('good', 'Adj')]
[('Cat', 'Noun'), ('eats', 'Verb'), ('meat', 'Noun')]
[('Cai', 'gay')]


In [43]:
get_token_POS(train_esp)

[[('Melbourne', 'NP'),
  ('(', 'Fpa'),
  ('Australia', 'NP'),
  (')', 'Fpt'),
  (',', 'Fc'),
  ('25', 'Z'),
  ('may', 'NC'),
  ('(', 'Fpa'),
  ('EFE', 'NC'),
  (')', 'Fpt'),
  ('.', 'Fp')],
 [('-', 'Fg')],
 [('El', 'DA'),
  ('Abogado', 'NC'),
  ('General', 'AQ'),
  ('del', 'SP'),
  ('Estado', 'NC'),
  (',', 'Fc'),
  ('Daryl', 'VMI'),
  ('Williams', 'NC'),
  (',', 'Fc'),
  ('subrayó', 'VMI'),
  ('hoy', 'RG'),
  ('la', 'DA'),
  ('necesidad', 'NC'),
  ('de', 'SP'),
  ('tomar', 'VMN'),
  ('medidas', 'NC'),
  ('para', 'SP'),
  ('proteger', 'VMN'),
  ('al', 'SP'),
  ('sistema', 'NC'),
  ('judicial', 'AQ'),
  ('australiano', 'AQ'),
  ('frente', 'RG'),
  ('a', 'SP'),
  ('una', 'DI'),
  ('página', 'NC'),
  ('de', 'SP'),
  ('internet', 'NC'),
  ('que', 'PR'),
  ('imposibilita', 'VMI'),
  ('el', 'DA'),
  ('cumplimiento', 'NC'),
  ('de', 'SP'),
  ('los', 'DA'),
  ('principios', 'NC'),
  ('básicos', 'AQ'),
  ('de', 'SP'),
  ('la', 'DA'),
  ('Ley', 'NC'),
  ('.', 'Fp')],
 [('La', 'DA'),
  ('petición

## Codificació BIO

In [17]:
tagged_words = get_token_entity(train_esp)[3]
print(tagged_words)

[('La', 'O'), ('petición', 'O'), ('del', 'O'), ('Abogado', 'B-PER'), ('General', 'I-PER'), ('tiene', 'O'), ('lugar', 'O'), ('después', 'O'), ('de', 'O'), ('que', 'O'), ('un', 'O'), ('juez', 'O'), ('del', 'O'), ('Tribunal', 'B-ORG'), ('Supremo', 'I-ORG'), ('del', 'O'), ('estado', 'O'), ('de', 'O'), ('Victoria', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), ('se', 'O'), ('viera', 'O'), ('forzado', 'O'), ('a', 'O'), ('disolver', 'O'), ('un', 'O'), ('jurado', 'O'), ('popular', 'O'), ('y', 'O'), ('suspender', 'O'), ('el', 'O'), ('proceso', 'O'), ('ante', 'O'), ('el', 'O'), ('argumento', 'O'), ('de', 'O'), ('la', 'O'), ('defensa', 'O'), ('de', 'O'), ('que', 'O'), ('las', 'O'), ('personas', 'O'), ('que', 'O'), ('lo', 'O'), ('componían', 'O'), ('podían', 'O'), ('haber', 'O'), ('obtenido', 'O'), ('información', 'O'), ('sobre', 'O'), ('el', 'O'), ('acusado', 'O'), ('a', 'O'), ('través', 'O'), ('de', 'O'), ('la', 'O'), ('página', 'O'), ('CrimeNet', 'B-MISC'), ('.', 'O')]


In [16]:
def extract_entities(tagged_words, encoding='BIO'):
    """
    Extreu les entitats d'una llista de paraules etiquetades segons l'encoding especificat.
    
    Arguments:
        tagged_words: una llista de tuples (word, tag), on 'word' és una paraula del text i 'tag' és la seva etiqueta (BIO/BIOE/BIOW/IO).
        encoding: el tipus de codificació utilitzat per les etiquetes ('BIO', 'BIOW', 'BIOE', 'IO').
        
    Retorna:
        Una llista de tuples (start_index, end_index, entity_type) que representen les entitats trobades.
        'start_index' i 'end_index' són els índexs on comença i acaba l'entitat en la llista de paraules, i 'entity_type' és el tipus d'entitat.
    """

    entities = []  # Llista on guardarem les entitats trobades
    current_entity = []  # Guarda les paraules de l'entitat actual
    current_type = None  # Tipus de l'entitat actual
    current_start_index = None  # Índex d'inici de l'entitat actual

    for index, (word, tag) in enumerate(tagged_words):
        tag_type = None if tag == 'O' else tag[2:]

        if tag == 'O':
            if current_entity:
                entities.append((current_start_index, index - 1, current_type))
                current_entity = []
                current_type = None
            continue

        if encoding == 'IO':
            if tag_type != current_type:
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                current_entity = [word]
                current_start_index = index
                current_type = tag_type
            else:
                current_entity.append(word)
        else:
            tag_prefix = tag[:1]
            if tag_prefix in ['B', 'W']:  # Començament d'una nova entitat o entitat de paraula única
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                current_entity = [word]
                current_start_index = index
                current_type = tag_type
                if tag_prefix == 'W':  # Si és una entitat de paraula única, la tanquem immediatament
                    entities.append((current_start_index, index, current_type))
                    current_entity = []
                    current_type = None
            elif tag_prefix == 'I' and current_type == tag_type:
                current_entity.append(word)
            elif encoding == 'BIOE' and tag_prefix == 'E' and current_type == tag_type:
                current_entity.append(word)
                entities.append((current_start_index, index, current_type))
                current_entity = []
                current_type = None
            else:
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                    current_entity = []
                current_type = None

    if current_entity:
        entities.append((current_start_index, index, current_type))

    return entities

tagged_words = get_token_entity(train_esp)[3]

entities = extract_entities(tagged_words, "BIO")
print(entities)


[(3, 4, 'PER'), (13, 14, 'ORG'), (18, 18, 'LOC'), (20, 20, 'LOC'), (59, 59, 'MISC')]


## Crear un model amb la nova feature function i codificació BIO

In [112]:
print("Quina forma tenen les nostres dades: ",train_esp[0])

Quina forma tenen les nostres dades:  [('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


In [45]:
"""def get_clean_sentence(sentence):
    """
    Receives a list and returns the first position of every element, corresponing to the token
    """
    clean = []
    for element in sentence: 
        clean.append(element[0])

    return clean"""

In [53]:
"""
train_esp_bio = []

for sent in train_esp: 
    i = 0
    clean_sentence = get_clean_sentence(sent)
    whole_sentence = []
    for token, tag, bio in sent:
        whole_sentence.append([[token, feature_getter.get_features(clean_sentence, i)], bio])
        i = i  + 1
    train_esp_bio.append(whole_sentence)

print(train_esp_bio[0][0])"""

[['Melbourne', {'bias': 1.0, 'has_capitalization': True, 'has_digit': False, 'has_punctuation': False, 'all_caps': False, 'is_capitalized': True, 'prefix': 'Mel', 'suffix': 'rne'}], 'B-LOC']


In [56]:
"""test_esp_bio = []

for sent in test_esp: 
    i = 0
    clean_sentence = get_clean_sentence(sent)
    whole_sentence = []
    for token, tag, bio in sent:
        whole_sentence.append([[token, feature_getter.get_features(clean_sentence, i)], bio])
        i = i  + 1
    test_esp_bio.append(whole_sentence)
print(test_esp_bio[0])
"""

[[['La', {'bias': 1.0, 'has_capitalization': True, 'has_digit': False, 'has_punctuation': False, 'all_caps': False, 'is_capitalized': True, 'prefix': 'La', 'suffix': 'La'}], 'B-LOC'], [['Coruña', {'bias': 1.0, 'has_capitalization': True, 'has_digit': False, 'has_punctuation': False, 'all_caps': False, 'is_capitalized': True, 'prefix': 'Cor', 'suffix': 'uña'}], 'I-LOC'], [[',', {'bias': 1.0, 'has_capitalization': False, 'has_digit': False, 'has_punctuation': True, 'all_caps': False, 'is_capitalized': False, 'prefix': ',', 'suffix': ','}], 'O'], [['23', {'bias': 1.0, 'has_capitalization': False, 'has_digit': True, 'has_punctuation': False, 'all_caps': False, 'is_capitalized': False, 'prefix': '23', 'suffix': '23'}], 'O'], [['may', {'bias': 1.0, 'has_capitalization': False, 'has_digit': False, 'has_punctuation': False, 'all_caps': False, 'is_capitalized': False, 'prefix': 'may', 'suffix': 'may'}], 'O'], [['(', {'bias': 1.0, 'has_capitalization': False, 'has_digit': False, 'has_punctuation

In [113]:

ct = CRFTagger(feature_func=feature_getter.get_features)

train_esp = get_token_entity(train_esp)

ct.train(train_esp, 'model.crf.tagger')




y_pred = ct.tag_sents(get_token(test_esp))


In [114]:

precision, recall, f1_score = evaluate_entities(y_real, y_pred, print_errors=False)
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1 Score: {f1_score:.6f}')


Precision: 0.710512
Recall: 0.710512
F1 Score: 0.710512


## Codificació IO

In [27]:
def bio_to_io(bio_tagged_sentences):
    """
    Converteix les etiquetes de múltiples frases des de la codificació BIO a IO.
    
    Arguments:
        bio_tagged_sentences: Una llista de llistes de tuples (word, tag) on 'tag' és en codificació BIO.
    
    Retorna:
        Una llista de llistes de tuples (word, io_tag) on 'io_tag' és en codificació IO per cada frase.
    """
    io_tagged_sentences = []
    
    for sentence in bio_tagged_sentences:
        io_tagged_sentence = []
        for word, tag in sentence:
            if tag.startswith('B-'):
                # Canvia B- per I-
                io_tagged_sentence.append((word, 'I-' + tag[2:]))
            elif tag.startswith('I-'):
                io_tagged_sentence.append((word, tag))
            else:
                # Manté les etiquetes 'O' tal com estan
                io_tagged_sentence.append((word, 'O'))
        io_tagged_sentences.append(io_tagged_sentence)
    
    return io_tagged_sentences


tagged_words = get_token_entity(train_esp)
tagged_words_io = bio_to_io(tagged_words)
print(f"BIO:{tagged_words[3]}")
print(f"IO:{tagged_words_io[3]}")

entities = extract_entities(tagged_words_io[3], "IO")
print(f"Entities IO: {entities}")

BIO:[('La', 'O'), ('petición', 'O'), ('del', 'O'), ('Abogado', 'B-PER'), ('General', 'I-PER'), ('tiene', 'O'), ('lugar', 'O'), ('después', 'O'), ('de', 'O'), ('que', 'O'), ('un', 'O'), ('juez', 'O'), ('del', 'O'), ('Tribunal', 'B-ORG'), ('Supremo', 'I-ORG'), ('del', 'O'), ('estado', 'O'), ('de', 'O'), ('Victoria', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), ('se', 'O'), ('viera', 'O'), ('forzado', 'O'), ('a', 'O'), ('disolver', 'O'), ('un', 'O'), ('jurado', 'O'), ('popular', 'O'), ('y', 'O'), ('suspender', 'O'), ('el', 'O'), ('proceso', 'O'), ('ante', 'O'), ('el', 'O'), ('argumento', 'O'), ('de', 'O'), ('la', 'O'), ('defensa', 'O'), ('de', 'O'), ('que', 'O'), ('las', 'O'), ('personas', 'O'), ('que', 'O'), ('lo', 'O'), ('componían', 'O'), ('podían', 'O'), ('haber', 'O'), ('obtenido', 'O'), ('información', 'O'), ('sobre', 'O'), ('el', 'O'), ('acusado', 'O'), ('a', 'O'), ('través', 'O'), ('de', 'O'), ('la', 'O'), ('página', 'O'), ('CrimeNet', 'B-MISC'), ('.', 'O')]
IO:[('La'

In [ ]:
'''from nltk.tag import CRFTagger
import pycrfsuite

ct = CRFTagger(feature_func=None)
#train and test sets without the postag
train_esp_bio = get_token_entity(train_esp)
test_esp_bio = get_token_entity(test_esp)
print(train_esp_bio[0])

#y_test is the true labels
y_test = [[iob for word, iob in sent] for sent in test_esp_bio]
print(y_test[0])

#train the model
ct.train(train_esp_bio, 'model.crf.tagger')

#predict the labels
y_pred = ct.tag_sents([[word for word, iob in sent] for sent in test_esp_bio])
y_pred = [[iob for word, iob in sent] for sent in y_pred]
print(y_pred[0])


#show the confusion matrix
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Flatten y_test and y_pred
y_test_flat = [iob for sent in y_test for iob in sent]
y_pred_flat = [iob for sent in y_pred for iob in sent]

# Generate confusion matrix
cm = confusion_matrix(y_test_flat, y_pred_flat)

# Visualize confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

#show the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test_flat, y_pred_flat))
ct.accuracy(test_esp_bio)'''

In [18]:
"""def evaluation(true_entities, pred_entities):
    """
    Avalua la predicció de les entitats amb precisió, record i F1-score.
    """
    true_positives = len(set(true_entities) & set(pred_entities))
    if true_positives == 0:
        return 0, 0, 0
    precision = true_positives / len(pred_entities)
    recall = true_positives / len(true_entities)
    f1_score = 2 * precision * recall / (precision + recall)
    return precision, recall, f1_score"""

In [ ]:
from nltk.tag import CRFTagger
import pycrfsuite

ct = CRFTagger(feature_func=feature_getter.get_features)

train_esp_bio = get_token_entity(train_esp)
test_esp_bio = get_token_entity(test_esp)

ct.train(train_esp_bio, 'model.crf.tagger')

# Test the model
test_pred = ct.tag_sents(get_token(test_esp))

